In [ ]:
import time
import pandas as pd
from fmu.sumo.explorer import Explorer
from fmu.sumo.explorer.objects.table import AggregatedTable
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Initialize Explorer

In [ ]:
sumo = Explorer("dev")

### Get Case

In [ ]:
# Get case by name (name is not guaranteed to be unique)
case = sumo.cases.filter(name="drogon_ahm-2023-02-22")[0]


### Finding info about tables connected to case

In [ ]:
tables = case.tables
print(f"Table names: {tables.names}")
print(f"Table tags: {tables.tagnames}")

### Finding info related to specific table name

In [ ]:
tables = tables.filter(name="summary")
print(f"Tagname check: {tables.tagnames}")

### The filter way

In [ ]:
table = case.tables.filter("summary", "eclipse", "iter-0", aggregation="collection", column="FOPT")
print(len(table))
print(table.tagnames)
print(table.names)
table = table[0]

frame = table.dataframe



### Access to global variables

Calculate CV (coefficient of variation) for all global variables to see which ones are varied the most

In [ ]:

globals = table["fmu"]["iteration"]["parameters"]["GLOBVAR"]
frame["REAL"] = frame["REAL"].astype(str)
for var_name in globals:
    series = pd.Series(globals[var_name].values())

    if series.mean() > 0.001:
        cv = series.std() / series.mean()
    else:
        cv = 0
    if cv > 0.01:
        print(f"{var_name}: {cv}")

    

### Add column with global var

In [ ]:
frame["RELPERM_INT_WO"] = frame["REAL"].replace(globals["RELPERM_INT_WO"])

In [ ]:
### Plot

In [ ]:
sns.lineplot(data=frame, x="DATE", y="FOPT", size="REAL", hue="RELPERM_INT_WO", legend=False)
plt.xticks(rotation=45)
plt.show()

### For even more user friendly access to summary data


In [ ]:
# Get case surfaces
summary = AggregatedTable(case, "summary", "eclipse", "iter-0")
summary.columns



### If you prefer arrow to pandas

In [ ]:
summary["FOPT"].arrowtable

### Inplace volumes

In [ ]:
# Get case surfaces
inplace = AggregatedTable(case, "geogrid", "vol", "iter-0")

inplace["STOIIP_OIL"].dataframe.groupby(["ZONE", "REAL"])["STOIIP_OIL"].agg("sum")["Therys"].plot(kind="hist")

### Summary speedtest

In [ ]:
start = time.perf_counter()
count = 0
for col_name in summary.columns[:20]:
    vector = summary[col_name]
    count += 1
print(f"{count} cols in total time: {time.perf_counter() - start: .1f} s")

### Inplace speedtest

In [ ]:
start = time.perf_counter()
count = 0
for col_name in inplace.columns[:20]:
    col = inplace[col_name]
    print(col)
    count += 1
print(f"{count} cols in total time: {time.perf_counter() - start: .1f} s")